In [ ]:
%%capture
!pip install datasets
!apt install git-lfs
!pip install transformers
!pip install sentencepiece 
!pip install rouge_score

In [ ]:
import numpy as np
from datasets import Dataset
import tensorflow as tf
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
nltk.download('punkt')

  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-9f74726fd5ea>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
REPO = "/content/drive/MyDrive/t5-ruspell-tune"  # Введите наазвание название репозитория
MODEL_NAME = "cointegrated/rut5-base" # Введите название выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8 
DATASET = 'UrukHan/t5-russian-spell_I'

In [ ]:
# датасет
data_1 = load_dataset(DATASET)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.cuda();

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--UrukHan--t5-russian-spell/snapshots/9e1e7dda2468aac53b8e4aa1ae30dc8fd9e9c47c/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--UrukHan--t5-russian-spell/snapshots/9e1e7dda2468aac53b8e4aa1ae30dc8fd9e9c47c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--UrukHan--t5-russian-spell/snapshots/9e1e7dda2468aac53b8e4aa1ae30dc8fd9e9c47c/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--UrukHan--t5-russian-spell/snapshots/9e1e7dda2468aac53b8e4aa1ae30dc8fd9e9c47c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--UrukHan--t5-russian-spell/snapshots/9e1e7dda2468aac53b8e4aa1ae30dc8fd9e9c47c/config.json
Model config T5Config {
  "_name_or_path": "UrukHan/t5-russ

In [ ]:
model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности

tokenizer config file saved in /tmp/tmpa1luw2j4/tokenizer_config.json
Special tokens file saved in /tmp/tmpa1luw2j4/special_tokens_map.json
Copy vocab file to /tmp/tmpa1luw2j4/spiece.model
Uploading the following files to igorktech/t5-ruspell-tune: tokenizer_config.json,spiece.model,special_tokens_map.json,tokenizer.json


CommitInfo(commit_url='https://huggingface.co/igorktech/t5-ruspell-tune/commit/7616b324052f6802b39d5ccb04597e42d1927f96', commit_message='Upload tokenizer', commit_description='', oid='7616b324052f6802b39d5ccb04597e42d1927f96', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import pandas as pd
# кастомный датасет
train_csv = pd.read_csv('/content/train.csv',sep = ",",encoding_errors='ignore')

In [ ]:
input_data = list(train_csv.iloc[0:50000, 0].values)
output_data = list(train_csv.iloc[0:50000, 1].values)
task_prefix = "Spell correct: "
input_sequences = input_data
encoding = tokenizer( [task_prefix + sequence for sequence in input_sequences], padding="longest", \
                     max_length=MAX_INPUT, truncation=True, return_tensors="pt", )
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
target_encoding = tokenizer(output_data, padding='max_length', max_length=MAX_OUTPUT, truncation=True)
labels = target_encoding.input_ids
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100
data = Dataset.from_pandas(pd.DataFrame({'input_ids': list(np.array(input_ids)), 'attention_mask': list(np.array(attention_mask)), \
                                         'labels': list(np.array(labels))}))
data = data.train_test_split(0.01)

corrupted_text  \
0                  Считает, что ссожет ить вечно!   
1                 20 миллионов и ни ппнни меньше!   
2                                - Но и мы умрём.   
3                                Ты вобще молоцд.   
4                     Вы имеетн в виду силу ьога?   
...                                           ...   
1480415    У омего папы был такой, только поольш.   
1480416                    Нкито там не работает.   
1480417                      Пистон обивки багаэи   
1480418        Макаронные езделмя Просто Спагетти   
1480419  Вы дьлжны найти способ aтклчить энергию.   

                                      correct_text  category  
0                  Считает, что сможет жить вечно!   phrases  
1                  20 миллионов и ни пенни меньше!   phrases  
2                                 - Но и мы умрём.   phrases  
3                                Ты вобще молодец.   phrases  
4                      Вы имеете в виду силу бога?   phrases  
...                                            ...       ...  
1480415   У моего папы был такой, только побольше.   phrases  
1480416                     Никто там не работает.   phrases  
1480417                      Пистон обивки багажни  products  
1480418         Макаронные изделия Просто Спагетти  products  
1480419  Вы должны найти способ отключить энергию.   phrases  

[1480420 rows x 3 columns]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 49500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [ ]:
data_1

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 67500
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1282500
    })
})

In [ ]:
from datasets import concatenate_datasets

In [ ]:
# создаем тренировочный датасет
data_train = data['train']#concatenate_datasets([data['train'], data_1['train']])
data_test =data['test'] #concatenate_datasets([data['test'], data_1['test']])

In [ ]:
data_test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 67900
})

In [ ]:
train = data_train
test = data_test#.train_test_split(0.02)['test']  # Уменьшим тестовую выборку чтоб не ждать долго расчет ошибок между эпохами

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #return_tensors="tf"

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  # Rouge expects a newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  # Extract a few results
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  
  # Add mean generated length
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
  output_dir = REPO,
  #overwrite_output_dir=True,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  eval_steps=5000,
  save_steps=5000,
  num_train_epochs=1,
  predict_with_generate=True,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  fp16=False, # True не всегда подходит
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  #logging_dir='logs',
  push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Выберем вручную оптимизатор. Т5 в оригинальной архитектуре использует Адафактор оптимизатор
optimizer = Adafactor(
    model.parameters(),
    lr=5e-5, # можно 1e-5
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

In [ ]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = train,
  eval_dataset = test,
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics
)

Cloning https://huggingface.co/igorktech/t5-ruspell-tune into local empty directory.


Download file spiece.model:   2%|1         | 15.4k/980k [00:00<?, ?B/s]

Using cuda_amp half precision backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 49500
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6188
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to t5-ruspell-tune/checkpoint-5000
Configuration saved in t5-ruspell-tune/checkpoint-5000/config.json
Model weights saved in t5-ruspell-tune/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in t5-ruspell-tune/checkpoint-5000/tokenizer_config.json
Special tokens file saved in t5-ruspell-tune/checkpoint-5000/special_tokens_map.json
Copy vocab file to t5-ruspell-tune/checkpoint-5000/spiece.model
tokenizer config file saved in t5-ruspell-tune/tokenizer_config.json
Special tokens file saved in t5-ruspell-tune/special_tokens_map.json
Copy vocab file to t5-ruspell-tune/spiece.model


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6188, training_loss=0.9073836505990993, metrics={'train_runtime': 2273.9117, 'train_samples_per_second': 21.769, 'train_steps_per_second': 2.721, 'total_flos': 2296080483840000.0, 'train_loss': 0.9073836505990993, 'epoch': 1.0})

In [ ]:
model.save_pretrained(REPO)
tokenizer.save_pretrained(REPO)

Configuration saved in t5-ruspell-tune/config.json
Model weights saved in t5-ruspell-tune/pytorch_model.bin
tokenizer config file saved in t5-ruspell-tune/tokenizer_config.json
Special tokens file saved in t5-ruspell-tune/special_tokens_map.json
Copy vocab file to t5-ruspell-tune/spiece.model


('t5-ruspell-tune/tokenizer_config.json',
 't5-ruspell-tune/special_tokens_map.json',
 't5-ruspell-tune/spiece.model',
 't5-ruspell-tune/added_tokens.json',
 't5-ruspell-tune/tokenizer.json')

In [ ]:
import pandas as pd

tokenizer = T5TokenizerFast.from_pretrained(REPO)
model = AutoModelForSeq2SeqLM.from_pretrained(REPO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.cuda();
# Входные данные
input_sequences = ['сеглдыя хорош ден', 'когд а вы прдет к нам в госи']
private_test_predicts = []
task_prefix = 'Spell correct: '
if type(input_sequences) != list: input_sequences = [input_sequences]
encoded = tokenizer(
  [task_prefix + sequence for sequence in input_sequences],
  padding="longest",
  max_length=256,
  truncation=True,
  return_tensors="pt",
    )

predicts = model.to(device).generate(**encoded.to(device))
private_test_predicts.append(tokenizer.batch_decode(predicts, skip_special_tokens=True))
# print(predicts)
print(private_test_predicts)

df = pd.DataFrame({'text': []})
for chunk in private_test_predicts:
    df = df.append(pd.DataFrame({'text': chunk}))
df.to_csv('private_test_lase.csv', header=None, index=None)

df.head(10)
# assert(df.shape[0] == private_test.shape[0])
# private_test.shape[0]

from IPython.display import FileLink

df.to_csv('private_test_submit_final.csv', index=False, header=None)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[['Сейчас хороший ден', 'Когда а вы придет к нам в гости.']]
